In [1]:
import cv2
import os
import mediapipe as mp

# Initialize MediaPipe Holistic, Hands, and Face
mp_holistic = mp.solutions.holistic
mp_hands = mp.solutions.hands
mp_face = mp.solutions.face_mesh

# Function to draw landmarks on frames
def draw_landmarks(image, output_path):
    # Convert the image color from BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image to extract landmarks
    with mp_holistic.Holistic(static_image_mode=True) as holistic, mp_hands.Hands(static_image_mode=True, max_num_hands=2) as hands, mp_face.FaceMesh(static_image_mode=True) as face:
        results_holistic = holistic.process(image)
        results_hands = hands.process(image)
        results_face = face.process(image)

        # Draw the landmarks on the image
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp.solutions.drawing_utils.draw_landmarks(image, results_holistic.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        mp.solutions.drawing_utils.draw_landmarks(image, results_holistic.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
        if results_hands.multi_hand_landmarks:
            for hand_landmarks in results_hands.multi_hand_landmarks:
                mp.solutions.drawing_utils.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Resize the image to 640x640
        image = cv2.resize(image, (640, 640))

    # Save the image with landmarks
    cv2.imwrite(output_path, image)

# Function to process frames
def process_frame(frame, output_frame_path):
    # Save the frame to the frames directory
    cv2.imwrite(output_frame_path, frame)

    # Draw landmarks on the frame and save to the landmarks directory
    draw_landmarks(frame, output_frame_path.replace('Extracted_frames', 'Extracted_landmarks'))

# Define your input directory
input_directory = "D:/VIDEO_DATASET_AlAmin"

# Define your output directories
output_root_frames = "D:/sign_custom_gg"
output_root_landmarks = "D:/sign_custom_landmarks"

# Iterate over classes
for class_name in os.listdir(input_directory):
    class_path = os.path.join(input_directory, class_name)

    # Skip non-directory entries
    if not os.path.isdir(class_path):
        continue

    # Iterate over videos in the class
    for video_name in os.listdir(class_path):
        video_path = os.path.join(class_path, video_name)

        # Open the video file
        cap = cv2.VideoCapture(video_path)

        # Get video properties
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Calculate frame indices to extract
        num_frames = 30
        frame_indices = [int(i * total_frames / num_frames) for i in range(num_frames)]

        # Generate output directories
        output_class_frames = os.path.join(output_root_frames, class_name, video_name.split('.')[0])
        output_class_landmarks = os.path.join(output_root_landmarks, class_name, video_name.split('.')[0])

        os.makedirs(output_class_frames, exist_ok=True)
        os.makedirs(output_class_landmarks, exist_ok=True)

        # Iterate over each frame index
        for idx in frame_indices:
            # Set the frame position
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)

            # Read the frame
            ret, frame = cap.read()
            if not ret:
                break

            # Generate output paths
            frame_filename = f"frame{idx + 1}.jpg"
            frame_path = os.path.join(output_class_frames, frame_filename)
            landmarks_filename = f"landmarks{idx + 1}.jpg"
            landmarks_path = os.path.join(output_class_landmarks, landmarks_filename)

            # Process frame
            process_frame(frame, frame_path)

        # Release the video capture object for the current video
        cap.release()
